# Colaborative Filtering

Collaborative filtering is a way recommendation systems filter information by using the preferences of other people. It uses the assumption that if person A has similar preferences to person B on items they have both reviewed, then person A is likely to have a similar preference to person B on an item only person B has reviewed.

Collaborative filtering is used by many recommendation systems in various fields, including music, shopping, financial data, and social networks and by various services (YouTube, Reddit, Last.fm). Any service that uses a recommendation system most likely employs collaborative filtering.

source: https://brilliant.org/wiki/collaborative-filtering/

## Memory-based CF

Memory-Based Collaborative Filtering approaches can be divided into two main sections: user-item (user-based) filtering and item-item (item-based) filtering. A user-item filtering takes a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. In contrast, item-item filtering will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations.

Item-Item Collaborative Filtering: “Users who liked this item also liked …”

User-Item Collaborative Filtering: “Users who are similar to you also liked …”

source: https://blog.cambridgespark.com/nowadays-recommender-systems-are-used-to-personalize-your-experience-on-the-web-telling-you-what-120f39b89c3c

## Model-baesd CF

Model-based Collaborative Filtering is based on matrix factorization (MF) which has received greater exposure, mainly as an unsupervised learning method for latent variable decomposition and dimensionality reduction. Matrix factorization is widely used for recommender systems where it can deal better with scalability and sparsity than Memory-based CF. The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items. When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization you can restructure the user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector. You fit this matrix to approximate your original matrix, as closely as possible, by multiplying the low-rank matrices together, which fills in the entries missing in the original matrix.

source: https://blog.cambridgespark.com/nowadays-recommender-systems-are-used-to-personalize-your-experience-on-the-web-telling-you-what-120f39b89c3c

## Task

In this exercise, you shall work with various collaborative filtering approaches. Specifically, you shall compare

* User based filtering
* Item based filtering
* One other model based filtering approach

You can reuse existing libraries and code examples (if you do so, please properly quote the origin, otherwise it has to be considered plagiarism), and you shall compare their performance in two ways

* Effectiveness of the recommendation on a supplied training set.
* Efficiency of the recommendation (i.e. runtime).

The dataset to be used is the MovieLens dataset. You shall first work with the smallest version available, with 100k ratings, at https://grouplens.org/datasets/movielens/100k/

To ensure that we can compare the results across all your peers in the course, you shall proceed as follows

* Split the dataset into 80:20 training:test set, after shuffling the data. 
* For evaluation of effectiveness, we will utilise MSE.

Your solution shall include the code, and a report on your findings - which methods worked well in regards to effectiveness and efficiency? Are the result in general usable?

After the first step on the 100k database, obtain the next bigger version (1M), and just test your algorithms for effectiveness - do the methods scale to the increased size?

## Dataset description

For this exercise 2 movielense datasets were used: 100k (https://grouplens.org/datasets/movielens/100k/) and 1M (https://grouplens.org/datasets/movielens/1M/)

MovieLens data sets were collected by the GroupLens Research Project at the University of Minnesota.

The 100k data set consists of:
* 100,000 ratings (1-5) 
* from 943 users 
* on 1682 movies.

The 1m data set consists of:
* 1,000,209 ratings (1-5) 
* from 6,040 users 
* on 3,952 movies.  

The data was collected through the MovieLens web site (movielens.umn.edu) during the seven-month period from September 19th, 1997 through April 22nd, 1998. This data has been cleaned up - users who had less than 20 ratings or did not have complete demographic information were removed from this data set.

sources: http://files.grouplens.org/datasets/movielens/ml-100k-README.txt http://files.grouplens.org/datasets/movielens/ml-1m-README.txt


In [12]:
import pandas as pd
import numpy as np

from hashlib import md5
from zipfile import ZipFile
import urllib.request
import os

data_attributes=["user_id","item_id","rating","timestamp"]

def get_content_file(url):
    h = md5(url.encode()).hexdigest()
    path = ".tmp/" + h
    file_path = path + "/data.zip"
    if not os.path.exists(file_path):
        if not os.path.exists(path):
            os.makedirs(path)
        urllib.request.urlretrieve(url, file_path)
    return file_path

file_path = get_content_file("http://files.grouplens.org/datasets/movielens/ml-1m.zip")
with ZipFile(file_path).open("ml-1m/ratings.dat") as decompressed_file:
    df_1m = pd.read_csv(
        decompressed_file,
        names=data_attributes,
        sep="::"
    )

file_path = get_content_file("http://files.grouplens.org/datasets/movielens/ml-100k.zip")
with ZipFile(file_path).open("ml-100k/u.data") as decompressed_file:
    df_100k = pd.read_csv(
        decompressed_file,
        names=data_attributes,
        sep="\t"
    )

In [69]:
from sklearn.model_selection import train_test_split

def split_and_pivot_dataset(df, test_size):
    train_data, test_data = train_test_split(df, test_size=test_size)

    # create user-item matrix as pivot table
    train_data_pivot = train_data.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)\
        .reindex(sorted(df.user_id.unique()), axis=0, fill_value=0)\
        .reindex(sorted(df.item_id.unique()), axis=1, fill_value=0)

    # create testset
    test_data_pivot = test_data.pivot_table(index='user_id', columns='item_id', values='rating', fill_value=0)\
        .reindex(sorted(df.user_id.unique()), axis=0, fill_value=0)\
        .reindex(sorted(df.item_id.unique()), axis=1, fill_value=0)
    
    return (train_data_pivot, test_data_pivot)

# Split into train and test-set (20% = 80:20) and create "user x item = rating" pivot table
train_data_pivot, test_data_pivot = split_and_pivot_dataset(df_1m, 0.2)
(train_data_pivot.shape, test_data_pivot.shape)

((6040, 3706), (6040, 3706))

In [70]:
from sklearn.metrics.pairwise import pairwise_distances

def predict_user(ratings):
    # calculate pairwise distances for items to users with cosine metric
    similarity = pairwise_distances(train_data_pivot, metric="cosine")
    # calculate mean values 
    mean_user_rating = ratings.mean(axis=1)
    # normalize ratings
    rating_diff = (ratings - mean_user_rating[:, np.newaxis])
    # calculate user-item correlations
    return mean_user_rating[:, np.newaxis] + similarity.dot(rating_diff) / np.array([np.abs(similarity).sum(axis=1)]).T

user_prediction = predict_user(train_data_pivot)
user_prediction

array([[ 1.0180236 ,  0.20521667,  0.11805004, ..., -0.04492769,
        -0.05070511,  0.12639953],
       [ 1.09904824,  0.25906479,  0.17028461, ...,  0.00984377,
         0.00480964,  0.17962567],
       [ 1.03029924,  0.19528433,  0.10735116, ..., -0.05453308,
        -0.05981754,  0.11935182],
       ...,
       [ 1.0227343 ,  0.17795679,  0.08507883, ..., -0.07942739,
        -0.08546295,  0.09316322],
       [ 1.09277612,  0.2570656 ,  0.16787987, ...,  0.0034236 ,
        -0.00187697,  0.17576156],
       [ 1.2404037 ,  0.41610148,  0.32872287, ...,  0.16246638,
         0.15756822,  0.32905561]])

In [47]:
%%timeit
predict_user(train_data_pivot, user_similarity)

2.78 s ± 165 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
from sklearn.metrics.pairwise import pairwise_distances

def predict_item(ratings):
    # calculate pairwise distances for users to items with cosine metric
    similarity = pairwise_distances(train_data_pivot.transpose(), metric="cosine")
    # calculate item-item correlations
    return ratings.values.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

item_prediction = predict_item(train_data_pivot)
item_prediction

array([[0.03994096, 0.04597291, 0.04877483, ..., 0.0516772 , 0.0522447 ,
        0.04912193],
       [0.0881389 , 0.09520341, 0.09850234, ..., 0.10348557, 0.10546052,
        0.09878872],
       [0.03238386, 0.03634189, 0.03841949, ..., 0.04175259, 0.0428211 ,
        0.03973853],
       ...,
       [0.01628693, 0.01805726, 0.01847296, ..., 0.01890494, 0.01916679,
        0.01836872],
       [0.08680269, 0.09385945, 0.09675042, ..., 0.09851305, 0.10017828,
        0.09661184],
       [0.22606293, 0.24504202, 0.25076236, ..., 0.25092862, 0.25267385,
        0.24295449]])

In [60]:
%%timeit
predict_item(train_data_pivot)

3.49 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
from scipy.sparse.linalg import svds

def predict_svd(ratings):
    U, S, Vt = svds(ratings, k=20)
    Sd = np.diag(S)
    return np.dot(np.dot(U, Sd), Vt)

model_prediction = predict_svd(train_data_pivot.values.astype(float))
model_prediction

array([[ 3.03404884e+00,  6.34649733e-01, -9.40281604e-03, ...,
        -3.77338131e-03, -1.62436537e-02,  1.02307408e-01],
       [ 8.18947825e-01,  3.54068048e-01,  2.12335766e-01, ...,
         1.76069786e-02,  2.73976838e-02,  5.56844141e-02],
       [ 7.17533577e-01,  1.12967827e-01,  1.03349885e-01, ...,
        -2.44407009e-02, -2.67896375e-03, -4.12819454e-02],
       ...,
       [ 5.71280996e-01,  1.67304835e-02, -3.49634201e-02, ...,
        -7.58731702e-03, -3.43104651e-04, -3.50520619e-02],
       [ 8.27784594e-01,  2.23862533e-01,  2.08391561e-01, ...,
         3.52896740e-02,  1.35098252e-02,  9.73627717e-03],
       [ 1.41986585e+00,  2.27577113e-01, -3.36611543e-01, ...,
         5.91306194e-02,  7.45368305e-02,  3.36650894e-01]])

In [46]:
%%timeit
predict_svd(train_data_pivot.values.astype(float))

2.66 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def mse(predicion, groud_truth):
    predicion = predicion[groud_truth != 0].flatten()
    groud_truth = groud_truth[groud_truth != 0].flatten()
    
    return mean_squared_error(predicion, groud_truth)

{
    'user_prediction_mse': mse(user_prediction, test_data_pivot.values), 
    'item_prediction_mse':  mse(item_prediction, test_data_pivot.values),
    'model_prediction_mse': mse(model_prediction, test_data_pivot.values)
}

{'user_prediction_mse': 10.356130342203791,
 'item_prediction_mse': 12.321712953156142,
 'model_prediction_mse': 7.324933251922383}